In [1]:
from model.srgan import generator
from model.wdsr import wdsr_b
from model import resolve_single
import utils
import cv2
import numpy as np

# SRGAN
srgan_model = generator()
srgan_model.load_weights('weights/srgan/gan_generator.h5')

#WDSR
wdsr_model = wdsr_b(scale=4, num_res_blocks=32)
wdsr_model.load_weights('weights/wdsr/wdsr-b-32-x4.h5')

In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
scale = 30
width = int(frame.shape[1] * (scale/100))
height = int(frame.shape[0] * (scale/100))
dim = (width, height)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    crop = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    upscale = resolve_single(wdsr_model, crop).numpy()
    
    # Display the resulting frame
    cv2.imshow('input', frame)
    cv2.imshow('cropped', crop)
    cv2.imshow('srgan', upscale)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [7]:
cap = cv2.VideoCapture('demo/test-footage.mp4')

scale = 25
width = int(frame.shape[1] * (scale/100))
height = int(frame.shape[0] * (scale/100))
dim = (width, height)

fourcc = cv2.VideoWriter_fourcc(*'H264')
out = cv2.VideoWriter('demo/rick-cop2.mp4', fourcc, 25, dim)
ret, frame = cap.read()

while ret == True:
    b = cv2.resize(frame, dim, fx=0, fy=0, interpolation = cv2.INTER_CUBIC)
    cv2.imshow('lol', frame)
    out.write(b)
    ret, frame = cap.read()
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
out.release()
cv2.destroyAllWindows()

In [2]:
cap = cv2.VideoCapture('demo/rick.mp4')

if (cap.isOpened()):
    ret, frame = cap.read()
    upscale = resolve_single(srgan_model, frame)
    width = int(upscale.shape[1])
    height = int(upscale.shape[0])
    dim=(width,height)
    fourcc = cv2.VideoWriter_fourcc(*'H264')
    out = cv2.VideoWriter('demo/rick-upscale.mp4', fourcc, 25, dim)
    
    while (cap.isOpened()):
        ret, frame = cap.read()
        upscale = resolve_single(wdsr_model, frame).numpy()
        out.write(upscale)
        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

Num GPUs Available:  1
